# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=BaseModel.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 BaseModel

This notebook contains high-level classes to interact with Mango Markets.

These often depend on structure layouts in [Layouts](Layouts.ipynb).

The idea is to have one high-level class, with useful members and methods, that can allow the Solana blobs loaded by the layout to vary. So there could be a layouts.SOMETHING_V1 and a layouts.SOMETHING_V2, and the Something class here can load from both.

Code, in general, depends only on these classes, abstracting away the version of the layout used to load the data.


In [ ]:
import abc
import construct
import datetime
import enum
import json
import logging
import time
import typing

import Layouts as layouts

from decimal import Decimal
from pyserum.market import Market
from pyserum.open_orders_account import OpenOrdersAccount
from solana.account import Account
from solana.publickey import PublicKey
from solana.rpc.commitment import Single
from solana.rpc.types import MemcmpOpts, TokenAccountOpts, RPCMethod, RPCResponse
from spl.token.client import Token as SplToken
from spl.token.constants import TOKEN_PROGRAM_ID

from Constants import SOL_DECIMALS, SOL_MINT_ADDRESS, SYSTEM_PROGRAM_ADDRESS
from Context import Context
from Decoder import decode_binary, encode_int, encode_binary, encode_key


## Version enum

This is used to keep track of which version of the layout struct was used to load the data.


In [ ]:
class Version(enum.Enum):
    UNSPECIFIED = 0
    V1 = 1
    V2 = 2
    V3 = 3
    V4 = 4
    V5 = 5


## InstructionType enum

This `enum` encapsulates all current Mango Market instruction variants.


In [ ]:
class InstructionType(enum.IntEnum):
    InitMangoGroup = 0
    InitMarginAccount = 1
    Deposit = 2
    Withdraw = 3
    Borrow = 4
    SettleBorrow = 5
    Liquidate = 6
    DepositSrm = 7
    WithdrawSrm = 8
    PlaceOrder = 9
    SettleFunds = 10
    CancelOrder = 11
    CancelOrderByClientId = 12
    ChangeBorrowLimit = 13
    PlaceAndSettle = 14
    ForceCancelOrders = 15
    PartialLiquidate = 16

    def __str__(self):
        return self.name


## Internal functions

These aren't published, they're really just for internal use. If they become more widely used we can move them to a better place.

In [ ]:
def _split_list_into_chunks(to_chunk: typing.List, chunk_size: int = 100) -> typing.List[typing.List]:
    chunks = []
    start = 0
    while start < len(to_chunk):
        chunk = to_chunk[start:start + chunk_size]
        chunks += [chunk]
        start += chunk_size
    return chunks


## AccountInfo class


In [ ]:
class AccountInfo:
    def __init__(self, address: PublicKey, executable: bool, lamports: Decimal, owner: PublicKey, rent_epoch: Decimal, data: bytes):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.address: PublicKey = address
        self.executable: bool = executable
        self.lamports: Decimal = lamports
        self.owner: PublicKey = owner
        self.rent_epoch: Decimal = rent_epoch
        self.data: bytes = data

    def encoded_data(self) -> typing.List:
        return encode_binary(self.data)

    def __str__(self) -> str:
        return f"""« AccountInfo [{self.address}]:
    Owner: {self.owner}
    Executable: {self.executable}
    Lamports: {self.lamports}
    Rent Epoch: {self.rent_epoch}
»"""

    def __repr__(self) -> str:
        return f"{self}"

    @staticmethod
    def load(context: Context, address: PublicKey) -> typing.Optional["AccountInfo"]:
        response: RPCResponse = context.client.get_account_info(address)
        result = context.unwrap_or_raise_exception(response)
        if result["value"] is None:
            return None

        return AccountInfo._from_response_values(result["value"], address)

    @staticmethod
    def load_multiple(context: Context, addresses: typing.List[PublicKey], chunk_size: int = 100, sleep_between_calls: float = 0.0) -> typing.List["AccountInfo"]:
        # This is a tricky one to get right.
        # Some errors this can generate:
        #  413 Client Error: Payload Too Large for url
        #  Error response from server: 'Too many inputs provided; max 100', code: -32602
        address_strings: typing.List[str] = list(map(PublicKey.__str__, addresses))
        multiple: typing.List[AccountInfo] = []
        chunks = _split_list_into_chunks(address_strings, chunk_size)
        for counter, chunk in enumerate(chunks):
            response = context.client._provider.make_request(RPCMethod("getMultipleAccounts"), chunk)
            result = context.unwrap_or_raise_exception(response)
            response_value_list = zip(result["value"], addresses)
            multiple += list(map(lambda pair: AccountInfo._from_response_values(pair[0], pair[1]), response_value_list))
            if (sleep_between_calls > 0.0) and (counter < (len(chunks) - 1)):
                time.sleep(sleep_between_calls)

        return multiple

    @staticmethod
    def _from_response_values(response_values: typing.Dict[str, typing.Any], address: PublicKey) -> "AccountInfo":
        executable = bool(response_values["executable"])
        lamports = Decimal(response_values["lamports"])
        owner = PublicKey(response_values["owner"])
        rent_epoch = Decimal(response_values["rentEpoch"])
        data = decode_binary(response_values["data"])
        return AccountInfo(address, executable, lamports, owner, rent_epoch, data)

    @staticmethod
    def from_response(response: RPCResponse, address: PublicKey) -> "AccountInfo":
        return AccountInfo._from_response_values(response["result"]["value"], address)


## AddressableAccount class

Some of our most-used objects (like `Group` or `MarginAccount`) are accounts on Solana with packed data. When these are loaded, they're typically loaded by loading the `AccountInfo` and parsing it in an object-specific way.

It's sometimes useful to be able to treat these in a common fashion so we use `AddressableAccount` as a way of sharing common features and providing a common base.

In [ ]:
class AddressableAccount(metaclass=abc.ABCMeta):
    def __init__(self, account_info: AccountInfo):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.account_info = account_info

    @property
    def address(self) -> PublicKey:
        return self.account_info.address

    def __repr__(self) -> str:
        return f"{self}"


## SerumAccountFlags class

The Serum prefix is because there's also `MangoAccountFlags` for the Mango-specific flags.

In [ ]:
class SerumAccountFlags:
    def __init__(self, version: Version, initialized: bool, market: bool, open_orders: bool,
                 request_queue: bool, event_queue: bool, bids: bool, asks: bool, disabled: bool):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.version: Version = version
        self.initialized = initialized
        self.market = market
        self.open_orders = open_orders
        self.request_queue = request_queue
        self.event_queue = event_queue
        self.bids = bids
        self.asks = asks
        self.disabled = disabled

    @staticmethod
    def from_layout(layout: layouts.SERUM_ACCOUNT_FLAGS) -> "SerumAccountFlags":
        return SerumAccountFlags(Version.UNSPECIFIED, layout.initialized, layout.market,
                                 layout.open_orders, layout.request_queue, layout.event_queue,
                                 layout.bids, layout.asks, layout.disabled)

    def __str__(self) -> str:
        flags: typing.List[typing.Optional[str]] = []
        flags += ["initialized" if self.initialized else None]
        flags += ["market" if self.market else None]
        flags += ["open_orders" if self.open_orders else None]
        flags += ["request_queue" if self.request_queue else None]
        flags += ["event_queue" if self.event_queue else None]
        flags += ["bids" if self.bids else None]
        flags += ["asks" if self.asks else None]
        flags += ["disabled" if self.disabled else None]
        flag_text = " | ".join(flag for flag in flags if flag is not None) or "None"
        return f"« SerumAccountFlags: {flag_text} »"

    def __repr__(self) -> str:
        return f"{self}"


## MangoAccountFlags class

The Mango prefix is because there's also `SerumAccountFlags` for the standard Serum flags.

In [ ]:
class MangoAccountFlags:
    def __init__(self, version: Version, initialized: bool, group: bool, margin_account: bool, srm_account: bool):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.version: Version = version
        self.initialized = initialized
        self.group = group
        self.margin_account = margin_account
        self.srm_account = srm_account

    @staticmethod
    def from_layout(layout: layouts.MANGO_ACCOUNT_FLAGS) -> "MangoAccountFlags":
        return MangoAccountFlags(Version.UNSPECIFIED, layout.initialized, layout.group,
                                 layout.margin_account, layout.srm_account)

    def __str__(self) -> str:
        flags: typing.List[typing.Optional[str]] = []
        flags += ["initialized" if self.initialized else None]
        flags += ["group" if self.group else None]
        flags += ["margin_account" if self.margin_account else None]
        flags += ["srm_account" if self.srm_account else None]
        flag_text = " | ".join(flag for flag in flags if flag is not None) or "None"
        return f"« MangoAccountFlags: {flag_text} »"

    def __repr__(self) -> str:
        return f"{self}"


## Index class

In [ ]:
class Index:
    def __init__(self, version: Version, last_update: datetime.datetime, borrow: Decimal, deposit: Decimal):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.version: Version = version
        self.last_update: datetime.datetime = last_update
        self.borrow: Decimal = borrow
        self.deposit: Decimal = deposit

    @staticmethod
    def from_layout(layout: layouts.INDEX, decimals: Decimal) -> "Index":
        borrow = layout.borrow / Decimal(10 ** decimals)
        deposit = layout.deposit / Decimal(10 ** decimals)
        return Index(Version.UNSPECIFIED, layout.last_update, borrow, deposit)

    def __str__(self) -> str:
        return f"« Index [{self.last_update}]: Borrow: {self.borrow:,.8f}, Deposit: {self.deposit:,.8f} »"

    def __repr__(self) -> str:
        return f"{self}"


## AggregatorConfig class

In [ ]:
class AggregatorConfig:
    def __init__(self, version: Version, description: str, decimals: Decimal, restart_delay: Decimal,
                 max_submissions: Decimal, min_submissions: Decimal, reward_amount: Decimal,
                 reward_token_account: PublicKey):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.version: Version = version
        self.description: str = description
        self.decimals: Decimal = decimals
        self.restart_delay: Decimal = restart_delay
        self.max_submissions: Decimal = max_submissions
        self.min_submissions: Decimal = min_submissions
        self.reward_amount: Decimal = reward_amount
        self.reward_token_account: PublicKey = reward_token_account

    @staticmethod
    def from_layout(layout: layouts.AGGREGATOR_CONFIG) -> "AggregatorConfig":
        return AggregatorConfig(Version.UNSPECIFIED, layout.description, layout.decimals,
                                layout.restart_delay, layout.max_submissions, layout.min_submissions,
                                layout.reward_amount, layout.reward_token_account)

    def __str__(self) -> str:
        return f"« AggregatorConfig: '{self.description}', Decimals: {self.decimals} [restart delay: {self.restart_delay}], Max: {self.max_submissions}, Min: {self.min_submissions}, Reward: {self.reward_amount}, Reward Account: {self.reward_token_account} »"

    def __repr__(self) -> str:
        return f"{self}"


## Round class

In [ ]:
class Round:
    def __init__(self, version: Version, id: Decimal, created_at: datetime.datetime, updated_at: datetime.datetime):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.version: Version = version
        self.id: Decimal = id
        self.created_at: datetime.datetime = created_at
        self.updated_at: datetime.datetime = updated_at

    @staticmethod
    def from_layout(layout: layouts.ROUND) -> "Round":
        return Round(Version.UNSPECIFIED, layout.id, layout.created_at, layout.updated_at)

    def __str__(self) -> str:
        return f"« Round[{self.id}], Created: {self.updated_at}, Updated: {self.updated_at} »"

    def __repr__(self) -> str:
        return f"{self}"


## Answer class

In [ ]:
class Answer:
    def __init__(self, version: Version, round_id: Decimal, median: Decimal, created_at: datetime.datetime, updated_at: datetime.datetime):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.version: Version = version
        self.round_id: Decimal = round_id
        self.median: Decimal = median
        self.created_at: datetime.datetime = created_at
        self.updated_at: datetime.datetime = updated_at

    @staticmethod
    def from_layout(layout: layouts.ANSWER) -> "Answer":
        return Answer(Version.UNSPECIFIED, layout.round_id, layout.median, layout.created_at, layout.updated_at)

    def __str__(self) -> str:
        return f"« Answer: Round[{self.round_id}], Median: {self.median:,.8f}, Created: {self.updated_at}, Updated: {self.updated_at} »"

    def __repr__(self) -> str:
        return f"{self}"


## Aggregator class

In [ ]:
class Aggregator(AddressableAccount):
    def __init__(self, account_info: AccountInfo, version: Version, config: AggregatorConfig,
                 initialized: bool, name: str, owner: PublicKey, round_: Round,
                 round_submissions: PublicKey, answer: Answer, answer_submissions: PublicKey):
        super().__init__(account_info)
        self.version: Version = version
        self.config: AggregatorConfig = config
        self.initialized: bool = initialized
        self.name: str = name
        self.owner: PublicKey = owner
        self.round: Round = round_
        self.round_submissions: PublicKey = round_submissions
        self.answer: Answer = answer
        self.answer_submissions: PublicKey = answer_submissions

    @property
    def price(self) -> Decimal:
        return self.answer.median / (10 ** self.config.decimals)

    @staticmethod
    def from_layout(layout: layouts.AGGREGATOR, account_info: AccountInfo, name: str) -> "Aggregator":
        config = AggregatorConfig.from_layout(layout.config)
        initialized = bool(layout.initialized)
        round_ = Round.from_layout(layout.round)
        answer = Answer.from_layout(layout.answer)

        return Aggregator(account_info, Version.UNSPECIFIED, config, initialized, name, layout.owner,
                          round_, layout.round_submissions, answer, layout.answer_submissions)

    @staticmethod
    def parse(context: Context, account_info: AccountInfo) -> "Aggregator":
        data = account_info.data
        if len(data) != layouts.AGGREGATOR.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.AGGREGATOR.sizeof()})")

        name = context.lookup_oracle_name(account_info.address)
        layout = layouts.AGGREGATOR.parse(data)
        return Aggregator.from_layout(layout, account_info, name)

    @staticmethod
    def load(context: Context, account_address: PublicKey):
        account_info = AccountInfo.load(context, account_address)
        if account_info is None:
            raise Exception(f"Aggregator account not found at address '{account_address}'")
        return Aggregator.parse(context, account_info)

    def __str__(self) -> str:
        return f"""
« Aggregator '{self.name}' [{self.version}]:
    Config: {self.config}
    Initialized: {self.initialized}
    Owner: {self.owner}
    Round: {self.round}
    Round Submissions: {self.round_submissions}
    Answer: {self.answer}
    Answer Submissions: {self.answer_submissions}
»
"""


## Token class

`Token` defines aspects common to every token.

In [ ]:
class Token:
    def __init__(self, symbol: str, name: str, mint: PublicKey, decimals: Decimal):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.symbol: str = symbol.upper()
        self.name: str = name
        self.mint: PublicKey = mint
        self.decimals: Decimal = decimals

    def round(self, value: Decimal) -> Decimal:
        return round(value, int(self.decimals))

    def symbol_matches(self, symbol: str) -> bool:
        return self.symbol.upper() == symbol.upper()

    @staticmethod
    def find_by_symbol(values: typing.List["Token"], symbol: str) -> "Token":
        found = [value for value in values if value.symbol_matches(symbol)]
        if len(found) == 0:
            raise Exception(f"Token '{symbol}' not found in token values: {values}")

        if len(found) > 1:
            raise Exception(f"Token '{symbol}' matched multiple tokens in values: {values}")

        return found[0]

    @staticmethod
    def find_by_mint(values: typing.List["Token"], mint: PublicKey) -> "Token":
        found = [value for value in values if value.mint == mint]
        if len(found) == 0:
            raise Exception(f"Token '{mint}' not found in token values: {values}")

        if len(found) > 1:
            raise Exception(f"Token '{mint}' matched multiple tokens in values: {values}")

        return found[0]

    # TokenMetadatas are equal if they have the same mint address.
    def __eq__(self, other):
        if hasattr(other, 'mint'):
            return self.mint == other.mint
        return False

    def __str__(self) -> str:
        return f"« Token '{self.name}' [{self.mint} ({self.decimals} decimals)] »"

    def __repr__(self) -> str:
        return f"{self}"


## SolToken object

It's sometimes handy to have a `Token` for SOL, but SOL isn't actually a token and can't appear in baskets. This object defines a special case for SOL.


In [ ]:
SolToken = Token("SOL", "Pure SOL", SOL_MINT_ADDRESS, SOL_DECIMALS)

## TokenLookup class

This class allows us to look up token symbols, names, mint addresses and decimals, all from our Solana static data.

The `_find_data_by_symbol()` is used here and later in the `SpotMarketLookup` class.

The static data is the [Solana token list](https://raw.githubusercontent.com/solana-labs/token-list/main/src/tokens/solana.tokenlist.json) provided by Serum.

You can load a `TokenLookup` class by something like:
```
with open("solana.tokenlist.json") as json_file:
    token_data = json.load(json_file)
    token_lookup = TokenLookup(token_data)
```

In [ ]:
def _find_data_by_symbol(symbol: str, token_data: typing.Dict) -> typing.Optional[typing.Dict]:
    for token in token_data["tokens"]:
        if token["symbol"] == symbol:
            return token
    return None


class TokenLookup:
    def __init__(self, token_data: typing.Dict) -> None:
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.token_data = token_data

    def find_by_symbol(self, symbol: str):
        found = _find_data_by_symbol(symbol, self.token_data)
        if found is not None:
            return Token(found["symbol"], found["name"], PublicKey(found["address"]), Decimal(found["decimals"]))

        return None

    def find_by_mint(self, mint: PublicKey):
        mint_string: str = str(mint)
        for token in self.token_data["tokens"]:
            if token["address"] == mint_string:
                return Token(token["symbol"], token["name"], PublicKey(token["address"]), Decimal(token["decimals"]))

        return None

    @staticmethod
    def default_lookups() -> "TokenLookup":
        with open("solana.tokenlist.json") as json_file:
            token_data = json.load(json_file)
            return TokenLookup(token_data)

## SpotMarket class

In [ ]:
class SpotMarket:
    def __init__(self, address: PublicKey, base: Token, quote: Token):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.address: PublicKey = address
        self.base: Token = base
        self.quote: Token = quote

    @property
    def name(self) -> str:
        return f"{self.base.symbol}/{self.quote.symbol}"

    def __str__(self) -> str:
        return f"« Market {self.name}: {self.address} »"

    def __repr__(self) -> str:
        return f"{self}"


## SpotMarketLookup class

This class allows us to look up Serum market addresses and tokens, all from our Solana static data.

The static data is the [Solana token list](https://raw.githubusercontent.com/solana-labs/token-list/main/src/tokens/solana.tokenlist.json) provided by Serum.

You can load a `SpotMarketLookup` class by something like:
```
with open("solana.tokenlist.json") as json_file:
    token_data = json.load(json_file)
    spot_market_lookup = SpotMarketLookup(token_data)
```
This uses the same data file as `TokenLookup` but it looks a lot more complicated. The main reason for this is that tokens are described in a list, whereas markets are optional child attributes of tokens.

To find a token, we can just go through the list.

To find a market, we need to split the market symbol into the two token symbols, go through the list, check if the item has the optional `extensions` attribute, and in there see if there is a name-value pair for the particular market we're interested in. Also, the current file only lists USDC and USDT markets, so that's all we can support this way.

In [ ]:
class SpotMarketLookup:
    def __init__(self, token_data: typing.Dict) -> None:
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.token_data: typing.Dict = token_data

    def find_by_symbol(self, symbol: str) -> typing.Optional[SpotMarket]:
        base_symbol, quote_symbol = symbol.split("/")
        base_data = _find_data_by_symbol(base_symbol, self.token_data)
        if base_data is None:
            self.logger.warning(f"Could not find data for base token '{base_symbol}'")
            return None
        base = Token(base_data["symbol"], base_data["name"], PublicKey(base_data["address"]), Decimal(base_data["decimals"]))

        quote_data = _find_data_by_symbol(quote_symbol, self.token_data)
        if quote_data is None:
            self.logger.warning(f"Could not find data for quote token '{quote_symbol}'")
            return None
        quote = Token(quote_data["symbol"], quote_data["name"], PublicKey(quote_data["address"]), Decimal(quote_data["decimals"]))

        if "extensions" not in base_data:
            self.logger.warning(f"No markets found for base token '{base.symbol}'.")
            return None

        if quote.symbol == "USDC":
            if "serumV3Usdc" not in base_data["extensions"]:
                self.logger.warning(f"No USDC market found for base token '{base.symbol}'.")
                return None

            market_address_string = base_data["extensions"]["serumV3Usdc"]
            market_address = PublicKey(market_address_string)
        elif quote.symbol == "USDT":
            if "serumV3Usdt" not in base_data["extensions"]:
                self.logger.warning(f"No USDT market found for base token '{base.symbol}'.")
                return None

            market_address_string = base_data["extensions"]["serumV3Usdt"]
            market_address = PublicKey(market_address_string)
        else:
            self.logger.warning(f"Could not find market with quote token '{quote.symbol}'. Only markets based on USDC or USDT are supported.")
            return None

        return SpotMarket(market_address, base, quote)

    def find_by_address(self, address: PublicKey) -> typing.Optional[SpotMarket]:
        address_string: str = str(address)
        for token_data in self.token_data["tokens"]:
            if "extensions" in token_data:
                if "serumV3Usdc" in token_data["extensions"]:
                    if token_data["extensions"]["serumV3Usdc"] == address_string:
                        market_address_string = token_data["extensions"]["serumV3Usdc"]
                        market_address = PublicKey(market_address_string)
                        base = Token(token_data["symbol"], token_data["name"], PublicKey(token_data["address"]), Decimal(token_data["decimals"]))
                        quote_data = _find_data_by_symbol("USDC", self.token_data)
                        if quote_data is None:
                            raise Exception("Could not load token data for USDC (which should always be present).")
                        quote = Token(quote_data["symbol"], quote_data["name"], PublicKey(quote_data["address"]), Decimal(quote_data["decimals"]))
                        return SpotMarket(market_address, base, quote)
                if "serumV3Usdt" in token_data["extensions"]:
                    if token_data["extensions"]["serumV3Usdt"] == address_string:
                        market_address_string = token_data["extensions"]["serumV3Usdt"]
                        market_address = PublicKey(market_address_string)
                        base = Token(token_data["symbol"], token_data["name"], PublicKey(token_data["address"]), Decimal(token_data["decimals"]))
                        quote_data = _find_data_by_symbol("USDT", self.token_data)
                        if quote_data is None:
                            raise Exception("Could not load token data for USDT (which should always be present).")
                        quote = Token(quote_data["symbol"], quote_data["name"], PublicKey(quote_data["address"]), Decimal(quote_data["decimals"]))
                        return SpotMarket(market_address, base, quote)
        return None

    @staticmethod
    def default_lookups() -> "SpotMarketLookup":
        with open("solana.tokenlist.json") as json_file:
            token_data = json.load(json_file)
            return SpotMarketLookup(token_data)

## BasketToken class

`BasketToken` defines aspects of `Token`s that are part of a `Group` basket.

In [ ]:
class BasketToken:
    def __init__(self, token: Token, vault: PublicKey, index: Index):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.token: Token = token
        self.vault: PublicKey = vault
        self.index: Index = index

    @staticmethod
    def find_by_symbol(values: typing.List["BasketToken"], symbol: str) -> "BasketToken":
        found = [value for value in values if value.token.symbol_matches(symbol)]
        if len(found) == 0:
            raise Exception(f"Token '{symbol}' not found in token values: {values}")

        if len(found) > 1:
            raise Exception(f"Token '{symbol}' matched multiple tokens in values: {values}")

        return found[0]

    @staticmethod
    def find_by_mint(values: typing.List["BasketToken"], mint: PublicKey) -> "BasketToken":
        found = [value for value in values if value.token.mint == mint]
        if len(found) == 0:
            raise Exception(f"Token '{mint}' not found in token values: {values}")

        if len(found) > 1:
            raise Exception(f"Token '{mint}' matched multiple tokens in values: {values}")

        return found[0]

    @staticmethod
    def find_by_token(values: typing.List["BasketToken"], token: Token) -> "BasketToken":
        return BasketToken.find_by_mint(values, token.mint)

    # BasketTokens are equal if they have the same underlying token.
    def __eq__(self, other):
        if hasattr(other, 'token'):
            return self.token == other.token
        return False

    def __str__(self) -> str:
        return f"""« BasketToken:
    {self.token}
    Vault: {self.vault}
    Index: {self.index}
»"""

    def __repr__(self) -> str:
        return f"{self}"


## TokenValue class

The `TokenValue` class is a simple way of keeping a token and value together, and displaying them nicely consistently.

In [ ]:
class TokenValue:
    def __init__(self, token: Token, value: Decimal):
        self.token = token
        self.value = value

    @staticmethod
    def fetch_total_value_or_none(context: Context, account_public_key: PublicKey, token: Token) -> typing.Optional["TokenValue"]:
        opts = TokenAccountOpts(mint=token.mint)

        token_accounts_response = context.client.get_token_accounts_by_owner(account_public_key, opts, commitment=context.commitment)
        token_accounts = token_accounts_response["result"]["value"]
        if len(token_accounts) == 0:
            return None

        total_value = Decimal(0)
        for token_account in token_accounts:
            result = context.client.get_token_account_balance(token_account["pubkey"], commitment=context.commitment)
            value = Decimal(result["result"]["value"]["amount"])
            decimal_places = result["result"]["value"]["decimals"]
            divisor = Decimal(10 ** decimal_places)
            total_value += value / divisor

        return TokenValue(token, total_value)

    @staticmethod
    def fetch_total_value(context: Context, account_public_key: PublicKey, token: Token) -> "TokenValue":
        value = TokenValue.fetch_total_value_or_none(context, account_public_key, token)
        if value is None:
            return TokenValue(token, Decimal(0))
        return value

    @staticmethod
    def report(reporter: typing.Callable[[str], None], values: typing.List["TokenValue"]) -> None:
        for value in values:
            reporter(f"{value.value:>18,.8f} {value.token.name}")

    @staticmethod
    def find_by_symbol(values: typing.List["TokenValue"], symbol: str) -> "TokenValue":
        found = [value for value in values if value.token.symbol_matches(symbol)]
        if len(found) == 0:
            raise Exception(f"Token '{symbol}' not found in token values: {values}")

        if len(found) > 1:
            raise Exception(f"Token '{symbol}' matched multiple tokens in values: {values}")

        return found[0]

    @staticmethod
    def find_by_mint(values: typing.List["TokenValue"], mint: PublicKey) -> "TokenValue":
        found = [value for value in values if value.token.mint == mint]
        if len(found) == 0:
            raise Exception(f"Token '{mint}' not found in token values: {values}")

        if len(found) > 1:
            raise Exception(f"Token '{mint}' matched multiple tokens in values: {values}")

        return found[0]

    @staticmethod
    def find_by_token(values: typing.List["TokenValue"], token: Token) -> "TokenValue":
        return TokenValue.find_by_mint(values, token.mint)

    @staticmethod
    def changes(before: typing.List["TokenValue"], after: typing.List["TokenValue"]) -> typing.List["TokenValue"]:
        changes: typing.List[TokenValue] = []
        for before_balance in before:
            after_balance = TokenValue.find_by_token(after, before_balance.token)
            result = TokenValue(before_balance.token, after_balance.value - before_balance.value)
            changes += [result]

        return changes

    def __str__(self) -> str:
        return f"« TokenValue: {self.value:>18,.8f} {self.token.name} »"

    def __repr__(self) -> str:
        return f"{self}"


## OwnedTokenValue class

Ties an owner and `TokenValue` together. This is useful in the `TransactionScout`, where token mints and values are given separate from the owner `PublicKey` - we can package them together in this `OwnedTokenValue` class.

In [ ]:
class OwnedTokenValue:
    def __init__(self, owner: PublicKey, token_value: TokenValue):
        self.owner = owner
        self.token_value = token_value

    @staticmethod
    def find_by_owner(values: typing.List["OwnedTokenValue"], owner: PublicKey) -> "OwnedTokenValue":
        found = [value for value in values if value.owner == owner]
        if len(found) == 0:
            raise Exception(f"Owner '{owner}' not found in: {values}")

        if len(found) > 1:
            raise Exception(f"Owner '{owner}' matched multiple tokens in: {values}")

        return found[0]

    @staticmethod
    def changes(before: typing.List["OwnedTokenValue"], after: typing.List["OwnedTokenValue"]) -> typing.List["OwnedTokenValue"]:
        changes: typing.List[OwnedTokenValue] = []
        for before_value in before:
            after_value = OwnedTokenValue.find_by_owner(after, before_value.owner)
            token_value = TokenValue(before_value.token_value.token, after_value.token_value.value - before_value.token_value.value)
            result = OwnedTokenValue(before_value.owner, token_value)
            changes += [result]

        return changes

    def __str__(self) -> str:
        return f"[{self.owner}]: {self.token_value}"

    def __repr__(self) -> str:
        return f"{self}"


## MarketMetadata class

In [ ]:
class MarketMetadata:
    def __init__(self, name: str, address: PublicKey, base: BasketToken, quote: BasketToken,
                 spot: SpotMarket, oracle: PublicKey, decimals: Decimal):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.name: str = name
        self.address: PublicKey = address
        self.base: BasketToken = base
        self.quote: BasketToken = quote
        self.spot: SpotMarket = spot
        self.oracle: PublicKey = oracle
        self.decimals: Decimal = decimals
        self._market = None

    def fetch_market(self, context: Context) -> Market:
        if self._market is None:
            self._market = Market.load(context.client, self.spot.address)

        return self._market

    def __str__(self) -> str:
        base = f"{self.base}".replace("\n", "\n    ")
        quote = f"{self.quote}".replace("\n", "\n    ")
        return f"""« Market '{self.name}' [{self.address}/{self.spot.address}]:
    Base: {base}
    Quote: {quote}
    Oracle: {self.oracle} ({self.decimals} decimals)
»"""

    def __repr__(self) -> str:
        return f"{self}"


## Group class

The `Group` class encapsulates the data for the Mango Group - the cross-margined basket of tokens with lending.

In [ ]:
class Group(AddressableAccount):
    def __init__(self, account_info: AccountInfo, version: Version, context: Context,
                 account_flags: MangoAccountFlags, basket_tokens: typing.List[BasketToken],
                 markets: typing.List[MarketMetadata],
                 signer_nonce: Decimal, signer_key: PublicKey, dex_program_id: PublicKey,
                 total_deposits: typing.List[Decimal], total_borrows: typing.List[Decimal],
                 maint_coll_ratio: Decimal, init_coll_ratio: Decimal, srm_vault: PublicKey,
                 admin: PublicKey, borrow_limits: typing.List[Decimal]):
        super().__init__(account_info)
        self.version: Version = version
        self.context: Context = context
        self.account_flags: MangoAccountFlags = account_flags
        self.basket_tokens: typing.List[BasketToken] = basket_tokens
        self.markets: typing.List[MarketMetadata] = markets
        self.signer_nonce: Decimal = signer_nonce
        self.signer_key: PublicKey = signer_key
        self.dex_program_id: PublicKey = dex_program_id
        self.total_deposits: typing.List[Decimal] = total_deposits
        self.total_borrows: typing.List[Decimal] = total_borrows
        self.maint_coll_ratio: Decimal = maint_coll_ratio
        self.init_coll_ratio: Decimal = init_coll_ratio
        self.srm_vault: PublicKey = srm_vault
        self.admin: PublicKey = admin
        self.borrow_limits: typing.List[Decimal] = borrow_limits

    @property
    def shared_quote_token(self) -> BasketToken:
        return self.basket_tokens[-1]

    @property
    def base_tokens(self) -> typing.List[BasketToken]:
        return self.basket_tokens[:-1]

    @staticmethod
    def from_layout(layout: construct.Struct, context: Context, account_info: AccountInfo, version: Version, token_lookup: TokenLookup = TokenLookup.default_lookups(), spot_market_lookup: SpotMarketLookup = SpotMarketLookup.default_lookups()) -> "Group":
        account_flags: MangoAccountFlags = MangoAccountFlags.from_layout(layout.account_flags)
        indexes = list(map(lambda pair: Index.from_layout(pair[0], pair[1]), zip(layout.indexes, layout.mint_decimals)))

        basket_tokens: typing.List[BasketToken] = []
        for index, token_address in enumerate(layout.tokens):
            static_token_data = token_lookup.find_by_mint(token_address)
            if static_token_data is None:
                raise Exception(f"Could not find token with mint '{token_address}'.")

            # We create a new Token object here specifically to force the use of our own decimals
            token = Token(static_token_data.symbol, static_token_data.name, token_address, layout.mint_decimals[index])
            basket_token = BasketToken(token, layout.vaults[index], indexes[index])
            basket_tokens += [basket_token]

        markets: typing.List[MarketMetadata] = []
        for index, market_address in enumerate(layout.spot_markets):
            spot_market = spot_market_lookup.find_by_address(market_address)
            if spot_market is None:
                raise Exception(f"Could not find spot market with address '{market_address}'.")

            base_token = BasketToken.find_by_mint(basket_tokens, spot_market.base.mint)
            quote_token = BasketToken.find_by_mint(basket_tokens, spot_market.quote.mint)

            market = MarketMetadata(spot_market.name, market_address, base_token, quote_token,
                                    spot_market, layout.oracles[index], layout.oracle_decimals[index])
            markets += [market]

        maint_coll_ratio = layout.maint_coll_ratio.quantize(Decimal('.01'))
        init_coll_ratio = layout.init_coll_ratio.quantize(Decimal('.01'))

        return Group(account_info, version, context, account_flags, basket_tokens, markets,
                     layout.signer_nonce, layout.signer_key, layout.dex_program_id, layout.total_deposits,
                     layout.total_borrows, maint_coll_ratio, init_coll_ratio, layout.srm_vault,
                     layout.admin, layout.borrow_limits)

    @staticmethod
    def parse(context: Context, account_info: AccountInfo) -> "Group":
        data = account_info.data
        if len(data) == layouts.GROUP_V1.sizeof():
            layout = layouts.GROUP_V1.parse(data)
            version: Version = Version.V1
        elif len(data) == layouts.GROUP_V2.sizeof():
            version = Version.V2
            layout = layouts.GROUP_V2.parse(data)
        else:
            raise Exception(f"Group data length ({len(data)}) does not match expected size ({layouts.GROUP_V1.sizeof()} or {layouts.GROUP_V2.sizeof()})")

        return Group.from_layout(layout, context, account_info, version)

    @staticmethod
    def load(context: Context):
        account_info = AccountInfo.load(context, context.group_id)
        if account_info is None:
            raise Exception(f"Group account not found at address '{context.group_id}'")
        return Group.parse(context, account_info)

    def price_index_of_token(self, token: Token) -> int:
        for index, existing in enumerate(self.basket_tokens):
            if existing.token == token:
                return index
        return -1

    def fetch_token_prices(self) -> typing.List[TokenValue]:
        started_at = time.time()

        # Note: we can just load the oracle data in a simpler way, with:
        #   oracles = map(lambda market: Aggregator.load(self.context, market.oracle), self.markets)
        # but that makes a network request for every oracle. We can reduce that to just one request
        # if we use AccountInfo.load_multiple() and parse the data ourselves.
        #
        # This seems to halve the time this function takes.
        oracle_addresses = list([market.oracle for market in self.markets])
        oracle_account_infos = AccountInfo.load_multiple(self.context, oracle_addresses)
        oracles = map(lambda oracle_account_info: Aggregator.parse(self.context, oracle_account_info), oracle_account_infos)
        prices = list(map(lambda oracle: oracle.price, oracles)) + [Decimal(1)]
        token_prices = []
        for index, price in enumerate(prices):
            token_prices += [TokenValue(self.basket_tokens[index].token, price)]

        time_taken = time.time() - started_at
        self.logger.info(f"Fetching prices complete. Time taken: {time_taken:.2f} seconds.")
        return token_prices

    @staticmethod
    def load_with_prices(context: Context) -> typing.Tuple["Group", typing.List[TokenValue]]:
        group = Group.load(context)
        prices = group.fetch_token_prices()
        return group, prices

    def fetch_balances(self, root_address: PublicKey) -> typing.List[TokenValue]:
        balances: typing.List[TokenValue] = []
        sol_balance = self.context.fetch_sol_balance(root_address)
        balances += [TokenValue(SolToken, sol_balance)]

        for basket_token in self.basket_tokens:
            balance = TokenValue.fetch_total_value(self.context, root_address, basket_token.token)
            balances += [balance]
        return balances

    # The old way of fetching ripe margin accounts was to fetch them all then inspect them to see
    # if they were ripe. That was a big performance problem - fetching all groups was quite a penalty.
    #
    # This is still how it's done in load_ripe_margin_accounts_v1().
    #
    # The newer mechanism is to look for the has_borrows flag in the MangoAccount. That should
    # mean fewer MarginAccounts need to be fetched.
    #
    # This newer method is implemented in load_ripe_margin_accounts_v2()
    def load_ripe_margin_accounts(self) -> typing.List["MarginAccount"]:
        if self.version == Version.V1:
            return self.load_ripe_margin_accounts_v1()
        else:
            return self.load_ripe_margin_accounts_v2()

    def load_ripe_margin_accounts_v2(self) -> typing.List["MarginAccount"]:
        started_at = time.time()

        filters = [
            # 'has_borrows' offset is: 8 + 32 + 32 + (5 * 16) + (5 * 16) + (4 * 32) + 1
            # = 361
            MemcmpOpts(
                offset=361,
                bytes=encode_int(1)
            ),
            MemcmpOpts(
                offset=layouts.MANGO_ACCOUNT_FLAGS.sizeof(),  # mango_group is just after the MangoAccountFlags, which is the first entry
                bytes=encode_key(self.address)
            )
        ]

        response = self.context.client.get_program_accounts(self.context.program_id, data_size=layouts.MARGIN_ACCOUNT_V2.sizeof(), memcmp_opts=filters, commitment=Single, encoding="base64")
        result = self.context.unwrap_or_raise_exception(response)
        margin_accounts = []
        open_orders_addresses = []
        for margin_account_data in result:
            address = PublicKey(margin_account_data["pubkey"])
            account = AccountInfo._from_response_values(margin_account_data["account"], address)
            margin_account = MarginAccount.parse(account)
            open_orders_addresses += margin_account.open_orders
            margin_accounts += [margin_account]

        self.logger.info(f"Fetched {len(margin_accounts)} V2 margin accounts to process.")

        # It looks like this will be more efficient - just specify only the addresses we
        # need, and install them.
        #
        # Unfortunately there's a limit of 100 for the getMultipleAccounts() RPC call,
        # and doing it repeatedly requires some pauses because of rate limits.
        #
        # It's quicker (so far) to bring back every openorders account for the group.
        #
        # open_orders_addresses = [oo for oo in open_orders_addresses if oo is not None]

        # open_orders_account_infos = AccountInfo.load_multiple(self.context, open_orders_addresses)
        # open_orders_account_infos_by_address = {key: value for key, value in [(str(account_info.address), account_info) for account_info in open_orders_account_infos]}

        # for margin_account in margin_accounts:
        #     margin_account.install_open_orders_accounts(self, open_orders_account_infos_by_address)

        # This just fetches every openorder account for the group.
        open_orders = OpenOrders.load_raw_open_orders_account_infos(self.context, self)
        self.logger.info(f"Fetched {len(open_orders)} openorders accounts.")
        for margin_account in margin_accounts:
            margin_account.install_open_orders_accounts(self, open_orders)

        prices = self.fetch_token_prices()
        ripe_accounts = MarginAccount.filter_out_unripe(margin_accounts, self, prices)

        time_taken = time.time() - started_at
        self.logger.info(f"Loading ripe 🥭 accounts complete. Time taken: {time_taken:.2f} seconds.")
        return ripe_accounts

    def load_ripe_margin_accounts_v1(self) -> typing.List["MarginAccount"]:
        started_at = time.time()

        margin_accounts = MarginAccount.load_all_for_group_with_open_orders(self.context, self.context.program_id, self)
        self.logger.info(f"Fetched {len(margin_accounts)} V1 margin accounts to process.")

        prices = self.fetch_token_prices()
        ripe_accounts = MarginAccount.filter_out_unripe(margin_accounts, self, prices)

        time_taken = time.time() - started_at
        self.logger.info(f"Loading ripe 🥭 accounts complete. Time taken: {time_taken:.2f} seconds.")
        return ripe_accounts

    def __str__(self) -> str:
        total_deposits = "\n        ".join(map(str, self.total_deposits))
        total_borrows = "\n        ".join(map(str, self.total_borrows))
        borrow_limits = "\n        ".join(map(str, self.borrow_limits))
        shared_quote_token = str(self.shared_quote_token).replace("\n", "\n        ")
        base_tokens = "\n        ".join([f"{tok}".replace("\n", "\n        ") for tok in self.base_tokens])
        markets = "\n        ".join([f"{mkt}".replace("\n", "\n        ") for mkt in self.markets])
        return f"""
« Group [{self.version}] {self.address}:
    Flags: {self.account_flags}
    Base Tokens:
        {base_tokens}
    Quote Token:
        {shared_quote_token}
    Markets:
        {markets}
    DEX Program ID: « {self.dex_program_id} »
    SRM Vault: « {self.srm_vault} »
    Admin: « {self.admin} »
    Signer Nonce: {self.signer_nonce}
    Signer Key: « {self.signer_key} »
    Initial Collateral Ratio: {self.init_coll_ratio}
    Maintenance Collateral Ratio: {self.maint_coll_ratio}
    Total Deposits:
        {total_deposits}
    Total Borrows:
        {total_borrows}
    Borrow Limits:
        {borrow_limits}
»
"""


## TokenAccount class

In [ ]:
class TokenAccount(AddressableAccount):
    def __init__(self, account_info: AccountInfo, version: Version, mint: PublicKey, owner: PublicKey, amount: Decimal):
        super().__init__(account_info)
        self.version: Version = version
        self.mint: PublicKey = mint
        self.owner: PublicKey = owner
        self.amount: Decimal = amount

    @staticmethod
    def create(context: Context, account: Account, token: Token):
        spl_token = SplToken(context.client, token.mint, TOKEN_PROGRAM_ID, account)
        owner = account.public_key()
        new_account_address = spl_token.create_account(owner)
        return TokenAccount.load(context, new_account_address)

    @staticmethod
    def fetch_all_for_owner_and_token(context: Context, owner_public_key: PublicKey, token: Token) -> typing.List["TokenAccount"]:
        opts = TokenAccountOpts(mint=token.mint)

        token_accounts_response = context.client.get_token_accounts_by_owner(owner_public_key, opts, commitment=context.commitment)

        all_accounts: typing.List[TokenAccount] = []
        for token_account_response in token_accounts_response["result"]["value"]:
            account_info = AccountInfo._from_response_values(token_account_response["account"], PublicKey(token_account_response["pubkey"]))
            token_account = TokenAccount.parse(account_info)
            all_accounts += [token_account]

        return all_accounts

    @staticmethod
    def fetch_largest_for_owner_and_token(context: Context, owner_public_key: PublicKey, token: Token) -> typing.Optional["TokenAccount"]:
        all_accounts = TokenAccount.fetch_all_for_owner_and_token(context, owner_public_key, token)

        largest_account: typing.Optional[TokenAccount] = None
        for token_account in all_accounts:
            if largest_account is None or token_account.amount > largest_account.amount:
                largest_account = token_account

        return largest_account

    @staticmethod
    def fetch_or_create_largest_for_owner_and_token(context: Context, account: Account, token: Token) -> "TokenAccount":
        all_accounts = TokenAccount.fetch_all_for_owner_and_token(context, account.public_key(), token)

        largest_account: typing.Optional[TokenAccount] = None
        for token_account in all_accounts:
            if largest_account is None or token_account.amount > largest_account.amount:
                largest_account = token_account

        if largest_account is None:
            return TokenAccount.create(context, account, token)

        return largest_account

    @staticmethod
    def from_layout(layout: layouts.TOKEN_ACCOUNT, account_info: AccountInfo) -> "TokenAccount":
        return TokenAccount(account_info, Version.UNSPECIFIED, layout.mint, layout.owner, layout.amount)

    @staticmethod
    def parse(account_info: AccountInfo) -> "TokenAccount":
        data = account_info.data
        if len(data) != layouts.TOKEN_ACCOUNT.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.TOKEN_ACCOUNT.sizeof()})")

        layout = layouts.TOKEN_ACCOUNT.parse(data)
        return TokenAccount.from_layout(layout, account_info)

    @staticmethod
    def load(context: Context, address: PublicKey) -> typing.Optional["TokenAccount"]:
        account_info = AccountInfo.load(context, address)
        if account_info is None or (len(account_info.data) != layouts.TOKEN_ACCOUNT.sizeof()):
            return None
        return TokenAccount.parse(account_info)

    def __str__(self) -> str:
        return f"« Token: Address: {self.address}, Mint: {self.mint}, Owner: {self.owner}, Amount: {self.amount} »"


## OpenOrders class


In [ ]:
class OpenOrders(AddressableAccount):
    def __init__(self, account_info: AccountInfo, version: Version, program_id: PublicKey,
                 account_flags: SerumAccountFlags, market: PublicKey, owner: PublicKey,
                 base_token_free: Decimal, base_token_total: Decimal, quote_token_free: Decimal,
                 quote_token_total: Decimal, free_slot_bits: Decimal, is_bid_bits: Decimal,
                 orders: typing.List[Decimal], client_ids: typing.List[Decimal],
                 referrer_rebate_accrued: Decimal):
        super().__init__(account_info)
        self.version: Version = version
        self.program_id: PublicKey = program_id
        self.account_flags: SerumAccountFlags = account_flags
        self.market: PublicKey = market
        self.owner: PublicKey = owner
        self.base_token_free: Decimal = base_token_free
        self.base_token_total: Decimal = base_token_total
        self.quote_token_free: Decimal = quote_token_free
        self.quote_token_total: Decimal = quote_token_total
        self.free_slot_bits: Decimal = free_slot_bits
        self.is_bid_bits: Decimal = is_bid_bits
        self.orders: typing.List[Decimal] = orders
        self.client_ids: typing.List[Decimal] = client_ids
        self.referrer_rebate_accrued: Decimal = referrer_rebate_accrued

    # Sometimes pyserum wants to take its own OpenOrdersAccount as a parameter (e.g. in settle_funds())
    def to_pyserum(self) -> OpenOrdersAccount:
        return OpenOrdersAccount.from_bytes(self.address, self.account_info.data)

    @staticmethod
    def from_layout(layout: layouts.OPEN_ORDERS, account_info: AccountInfo,
                    base_decimals: Decimal, quote_decimals: Decimal) -> "OpenOrders":
        account_flags = SerumAccountFlags.from_layout(layout.account_flags)
        program_id = account_info.owner

        base_divisor = 10 ** base_decimals
        quote_divisor = 10 ** quote_decimals
        base_token_free: Decimal = layout.base_token_free / base_divisor
        base_token_total: Decimal = layout.base_token_total / base_divisor
        quote_token_free: Decimal = layout.quote_token_free / quote_divisor
        quote_token_total: Decimal = layout.quote_token_total / quote_divisor
        nonzero_orders: typing.List[Decimal] = list([order for order in layout.orders if order != 0])
        nonzero_client_ids: typing.List[Decimal] = list([client_id for client_id in layout.client_ids if client_id != 0])

        return OpenOrders(account_info, Version.UNSPECIFIED, program_id, account_flags, layout.market,
                          layout.owner, base_token_free, base_token_total, quote_token_free, quote_token_total,
                          layout.free_slot_bits, layout.is_bid_bits, nonzero_orders, nonzero_client_ids,
                          layout.referrer_rebate_accrued)

    @staticmethod
    def parse(account_info: AccountInfo, base_decimals: Decimal, quote_decimals: Decimal) -> "OpenOrders":
        data = account_info.data
        if len(data) != layouts.OPEN_ORDERS.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.OPEN_ORDERS.sizeof()})")

        layout = layouts.OPEN_ORDERS.parse(data)
        return OpenOrders.from_layout(layout, account_info, base_decimals, quote_decimals)

    @staticmethod
    def load_raw_open_orders_account_infos(context: Context, group: Group) -> typing.Dict[str, AccountInfo]:
        filters = [
            MemcmpOpts(
                offset=layouts.SERUM_ACCOUNT_FLAGS.sizeof() + 37,
                bytes=encode_key(group.signer_key)
            )
        ]

        response = context.client.get_program_accounts(group.dex_program_id, data_size=layouts.OPEN_ORDERS.sizeof(), memcmp_opts=filters, commitment=Single, encoding="base64")
        account_infos = list(map(lambda pair: AccountInfo._from_response_values(pair[0], pair[1]), [(result["account"], PublicKey(result["pubkey"])) for result in response["result"]]))
        account_infos_by_address = {key: value for key, value in [(str(account_info.address), account_info) for account_info in account_infos]}
        return account_infos_by_address

    @staticmethod
    def load(context: Context, address: PublicKey, base_decimals: Decimal, quote_decimals: Decimal) -> "OpenOrders":
        open_orders_account = AccountInfo.load(context, address)
        if open_orders_account is None:
            raise Exception(f"OpenOrders account not found at address '{address}'")
        return OpenOrders.parse(open_orders_account, base_decimals, quote_decimals)

    @staticmethod
    def load_for_market_and_owner(context: Context, market: PublicKey, owner: PublicKey, program_id: PublicKey, base_decimals: Decimal, quote_decimals: Decimal):
        filters = [
            MemcmpOpts(
                offset=layouts.SERUM_ACCOUNT_FLAGS.sizeof() + 5,
                bytes=encode_key(market)
            ),
            MemcmpOpts(
                offset=layouts.SERUM_ACCOUNT_FLAGS.sizeof() + 37,
                bytes=encode_key(owner)
            )
        ]

        response = context.client.get_program_accounts(context.dex_program_id, data_size=layouts.OPEN_ORDERS.sizeof(), memcmp_opts=filters, commitment=Single, encoding="base64")
        accounts = list(map(lambda pair: AccountInfo._from_response_values(pair[0], pair[1]), [(result["account"], PublicKey(result["pubkey"])) for result in response["result"]]))
        return list(map(lambda acc: OpenOrders.parse(acc, base_decimals, quote_decimals), accounts))

    def __str__(self) -> str:
        orders = ", ".join(map(str, self.orders)) or "None"
        client_ids = ", ".join(map(str, self.client_ids)) or "None"

        return f"""« OpenOrders:
    Flags: {self.account_flags}
    Program ID: {self.program_id}
    Address: {self.address}
    Market: {self.market}
    Owner: {self.owner}
    Base Token: {self.base_token_free:,.8f} of {self.base_token_total:,.8f}
    Quote Token: {self.quote_token_free:,.8f} of {self.quote_token_total:,.8f}
    Referrer Rebate Accrued: {self.referrer_rebate_accrued}
    Orders:
        {orders}
    Client IDs:
        {client_ids}
»"""


## BalanceSheet class

In [ ]:
class BalanceSheet:
    def __init__(self, token: Token, liabilities: Decimal, settled_assets: Decimal, unsettled_assets: Decimal):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.token: Token = token
        self.liabilities: Decimal = liabilities
        self.settled_assets: Decimal = settled_assets
        self.unsettled_assets: Decimal = unsettled_assets

    @property
    def assets(self) -> Decimal:
        return self.settled_assets + self.unsettled_assets

    @property
    def value(self) -> Decimal:
        return self.assets - self.liabilities

    @property
    def collateral_ratio(self) -> Decimal:
        if self.liabilities == Decimal(0):
            return Decimal(0)
        return self.assets / self.liabilities

    def __str__(self) -> str:
        name = "«Unspecified»"
        if self.token is not None:
            name = self.token.name

        return f"""« BalanceSheet [{name}]:
    Assets :           {self.assets:>18,.8f}
    Settled Assets :   {self.settled_assets:>18,.8f}
    Unsettled Assets : {self.unsettled_assets:>18,.8f}
    Liabilities :      {self.liabilities:>18,.8f}
    Value :            {self.value:>18,.8f}
    Collateral Ratio : {self.collateral_ratio:>18,.2%}
»
"""

    def __repr__(self) -> str:
        return f"{self}"


## MarginAccount class


In [ ]:
class MarginAccount(AddressableAccount):
    def __init__(self, account_info: AccountInfo, version: Version, account_flags: MangoAccountFlags,
                 has_borrows: bool, mango_group: PublicKey, owner: PublicKey,
                 deposits: typing.List[Decimal], borrows: typing.List[Decimal],
                 open_orders: typing.List[PublicKey]):
        super().__init__(account_info)
        self.version: Version = version
        self.account_flags: MangoAccountFlags = account_flags
        self.has_borrows: bool = has_borrows
        self.mango_group: PublicKey = mango_group
        self.owner: PublicKey = owner
        self.deposits: typing.List[Decimal] = deposits
        self.borrows: typing.List[Decimal] = borrows
        self.open_orders: typing.List[PublicKey] = open_orders
        self.open_orders_accounts: typing.List[typing.Optional[OpenOrders]] = [None] * len(open_orders)

    @staticmethod
    def from_layout(layout: construct.Struct, account_info: AccountInfo, version: Version) -> "MarginAccount":
        if version == Version.V1:
            # This is an old-style margin account, with no borrows flag
            has_borrows = False
        else:
            # This is a new-style margin account where we can depend on the presence of the borrows flag
            has_borrows = bool(layout.has_borrows)

        account_flags: MangoAccountFlags = MangoAccountFlags.from_layout(layout.account_flags)
        deposits: typing.List[Decimal] = []
        for index, deposit in enumerate(layout.deposits):
            deposits += [deposit]

        borrows: typing.List[Decimal] = []
        for index, borrow in enumerate(layout.borrows):
            borrows += [borrow]

        return MarginAccount(account_info, version, account_flags, has_borrows, layout.mango_group,
                             layout.owner, deposits, borrows, list(layout.open_orders))

    @staticmethod
    def parse(account_info: AccountInfo) -> "MarginAccount":
        data = account_info.data
        if len(data) == layouts.MARGIN_ACCOUNT_V1.sizeof():
            layout = layouts.MARGIN_ACCOUNT_V1.parse(data)
            version: Version = Version.V1
        elif len(data) == layouts.MARGIN_ACCOUNT_V2.sizeof():
            version = Version.V2
            layout = layouts.MARGIN_ACCOUNT_V2.parse(data)
        else:
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.MARGIN_ACCOUNT_V1.sizeof()} or {layouts.MARGIN_ACCOUNT_V2.sizeof()})")

        return MarginAccount.from_layout(layout, account_info, version)

    @staticmethod
    def load(context: Context, margin_account_address: PublicKey, group: typing.Optional[Group] = None) -> "MarginAccount":
        account_info = AccountInfo.load(context, margin_account_address)
        if account_info is None:
            raise Exception(f"MarginAccount account not found at address '{margin_account_address}'")

        margin_account = MarginAccount.parse(account_info)
        if group is None:
            group_context = context.new_from_group_id(margin_account.mango_group)
            group = Group.load(group_context)
        margin_account.load_open_orders_accounts(context, group)
        return margin_account

    @staticmethod
    def load_all_for_group(context: Context, program_id: PublicKey, group: Group) -> typing.List["MarginAccount"]:
        filters = [
            MemcmpOpts(
                offset=layouts.MANGO_ACCOUNT_FLAGS.sizeof(),  # mango_group is just after the MangoAccountFlags, which is the first entry
                bytes=encode_key(group.address)
            )
        ]

        if group.version == Version.V1:
            parser = layouts.MARGIN_ACCOUNT_V1
        else:
            parser = layouts.MARGIN_ACCOUNT_V2

        response = context.client.get_program_accounts(program_id, data_size=parser.sizeof(), memcmp_opts=filters, commitment=Single, encoding="base64")
        margin_accounts = []
        for margin_account_data in response["result"]:
            address = PublicKey(margin_account_data["pubkey"])
            account = AccountInfo._from_response_values(margin_account_data["account"], address)
            margin_account = MarginAccount.parse(account)
            margin_accounts += [margin_account]
        return margin_accounts

    @staticmethod
    def load_all_for_group_with_open_orders(context: Context, program_id: PublicKey, group: Group) -> typing.List["MarginAccount"]:
        margin_accounts = MarginAccount.load_all_for_group(context, context.program_id, group)
        open_orders = OpenOrders.load_raw_open_orders_account_infos(context, group)
        for margin_account in margin_accounts:
            margin_account.install_open_orders_accounts(group, open_orders)

        return margin_accounts

    @staticmethod
    def load_all_for_owner(context: Context, owner: PublicKey, group: typing.Optional[Group] = None) -> typing.List["MarginAccount"]:
        if group is None:
            group = Group.load(context)

        mango_group_offset = layouts.MANGO_ACCOUNT_FLAGS.sizeof()  # mango_group is just after the MangoAccountFlags, which is the first entry.
        owner_offset = mango_group_offset + 32  # owner is just after mango_group in the layout, and it's a PublicKey which is 32 bytes.
        filters = [
            MemcmpOpts(
                offset=mango_group_offset,
                bytes=encode_key(group.address)
            ),
            MemcmpOpts(
                offset=owner_offset,
                bytes=encode_key(owner)
            )
        ]

        response = context.client.get_program_accounts(context.program_id, memcmp_opts=filters, commitment=Single, encoding="base64")
        margin_accounts = []
        for margin_account_data in response["result"]:
            address = PublicKey(margin_account_data["pubkey"])
            account = AccountInfo._from_response_values(margin_account_data["account"], address)
            margin_account = MarginAccount.parse(account)
            margin_account.load_open_orders_accounts(context, group)
            margin_accounts += [margin_account]
        return margin_accounts

    @classmethod
    def filter_out_unripe(cls, margin_accounts: typing.List["MarginAccount"], group: Group, prices: typing.List[TokenValue]) -> typing.List["MarginAccount"]:
        logger: logging.Logger = logging.getLogger(cls.__name__)

        nonzero: typing.List[MarginAccountMetadata] = []
        for margin_account in margin_accounts:
            balance_sheet = margin_account.get_balance_sheet_totals(group, prices)
            if balance_sheet.collateral_ratio > 0:
                balances = margin_account.get_intrinsic_balances(group)
                nonzero += [MarginAccountMetadata(margin_account, balance_sheet, balances)]
        logger.info(f"Of those {len(margin_accounts)}, {len(nonzero)} have a nonzero collateral ratio.")

        ripe_metadata = filter(lambda mam: mam.balance_sheet.collateral_ratio <= group.init_coll_ratio, nonzero)
        ripe_accounts = list(map(lambda mam: mam.margin_account, ripe_metadata))
        logger.info(f"Of those {len(nonzero)}, {len(ripe_accounts)} are ripe 🥭.")
        return ripe_accounts

    def load_open_orders_accounts(self, context: Context, group: Group) -> None:
        for index, oo in enumerate(self.open_orders):
            key = oo
            if key is not None:
                self.open_orders_accounts[index] = OpenOrders.load(context, key, group.basket_tokens[index].token.decimals, group.shared_quote_token.token.decimals)

    def install_open_orders_accounts(self, group: Group, all_open_orders_by_address: typing.Dict[str, AccountInfo]) -> None:
        for index, oo in enumerate(self.open_orders):
            key = str(oo)
            if key in all_open_orders_by_address:
                open_orders_account_info = all_open_orders_by_address[key]
                open_orders = OpenOrders.parse(open_orders_account_info,
                                               group.basket_tokens[index].token.decimals,
                                               group.shared_quote_token.token.decimals)
                self.open_orders_accounts[index] = open_orders

    def get_intrinsic_balance_sheets(self, group: Group) -> typing.List[BalanceSheet]:
        settled_assets: typing.List[Decimal] = [Decimal(0)] * len(group.basket_tokens)
        liabilities: typing.List[Decimal] = [Decimal(0)] * len(group.basket_tokens)
        for index, token in enumerate(group.basket_tokens):
            settled_assets[index] = token.index.deposit * self.deposits[index]
            liabilities[index] = token.index.borrow * self.borrows[index]

        unsettled_assets: typing.List[Decimal] = [Decimal(0)] * len(group.basket_tokens)
        for index, open_orders_account in enumerate(self.open_orders_accounts):
            if open_orders_account is not None:
                unsettled_assets[index] += open_orders_account.base_token_total
                unsettled_assets[-1] += open_orders_account.quote_token_total

        balance_sheets: typing.List[BalanceSheet] = []
        for index, token in enumerate(group.basket_tokens):
            balance_sheets += [BalanceSheet(token.token, liabilities[index],
                                            settled_assets[index], unsettled_assets[index])]

        return balance_sheets

    def get_priced_balance_sheets(self, group: Group, prices: typing.List[TokenValue]) -> typing.List[BalanceSheet]:
        priced: typing.List[BalanceSheet] = []
        balance_sheets = self.get_intrinsic_balance_sheets(group)
        for balance_sheet in balance_sheets:
            price = TokenValue.find_by_token(prices, balance_sheet.token)
            liabilities = balance_sheet.liabilities * price.value
            settled_assets = balance_sheet.settled_assets * price.value
            unsettled_assets = balance_sheet.unsettled_assets * price.value
            priced += [BalanceSheet(
                price.token,
                price.token.round(liabilities),
                price.token.round(settled_assets),
                price.token.round(unsettled_assets)
            )]

        return priced

    def get_balance_sheet_totals(self, group: Group, prices: typing.List[TokenValue]) -> BalanceSheet:
        liabilities = Decimal(0)
        settled_assets = Decimal(0)
        unsettled_assets = Decimal(0)

        balance_sheets = self.get_priced_balance_sheets(group, prices)
        for balance_sheet in balance_sheets:
            if balance_sheet is not None:
                liabilities += balance_sheet.liabilities
                settled_assets += balance_sheet.settled_assets
                unsettled_assets += balance_sheet.unsettled_assets

        # A BalanceSheet must have a token - it's a pain to make it a typing.Optional[Token].
        # So in this one case, we produce a 'fake' token whose symbol is a summary of all token
        # symbols that went into it.
        #
        # If this becomes more painful than typing.Optional[Token], we can go with making
        # Token optional.
        summary_name = "-".join([bal.token.name for bal in balance_sheets])
        summary_token = Token(summary_name, f"{summary_name} Summary", SYSTEM_PROGRAM_ADDRESS, Decimal(0))
        return BalanceSheet(summary_token, liabilities, settled_assets, unsettled_assets)

    def get_intrinsic_balances(self, group: Group) -> typing.List[TokenValue]:
        balance_sheets = self.get_intrinsic_balance_sheets(group)
        balances: typing.List[TokenValue] = []
        for index, balance_sheet in enumerate(balance_sheets):
            if balance_sheet.token is None:
                raise Exception(f"Intrinsic balance sheet with index [{index}] has no token.")
            balances += [TokenValue(balance_sheet.token, balance_sheet.value)]

        return balances

    def __str__(self) -> str:
        deposits = ", ".join([f"{item:,.8f}" for item in self.deposits])
        borrows = ", ".join([f"{item:,.8f}" for item in self.borrows])
        if all(oo is None for oo in self.open_orders_accounts):
            open_orders = f"{self.open_orders}"
        else:
            open_orders_unindented = f"{self.open_orders_accounts}"
            open_orders = open_orders_unindented.replace("\n", "\n    ")
        return f"""« MarginAccount: {self.address}
    Flags: {self.account_flags}
    Has Borrows: {self.has_borrows}
    Owner: {self.owner}
    Mango Group: {self.mango_group}
    Deposits: [{deposits}]
    Borrows: [{borrows}]
    Mango Open Orders: {open_orders}
»"""


## MarginAccountMetadata class

In [ ]:
class MarginAccountMetadata:
    def __init__(self, margin_account: MarginAccount, balance_sheet: BalanceSheet, balances: typing.List[TokenValue]):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.margin_account = margin_account
        self.balance_sheet = balance_sheet
        self.balances = balances

    @property
    def assets(self):
        return self.balance_sheet.assets

    @property
    def liabilities(self):
        return self.balance_sheet.liabilities

    @property
    def collateral_ratio(self):
        return self.balance_sheet.collateral_ratio


# Events

## LiquidationEvent

In [ ]:
class LiquidationEvent:
    def __init__(self, timestamp: datetime.datetime, liquidator_name: str, group_name: str, succeeded: bool, signature: str, wallet_address: PublicKey, margin_account_address: PublicKey, balances_before: typing.List[TokenValue], balances_after: typing.List[TokenValue]):
        self.timestamp: datetime.datetime = timestamp
        self.liquidator_name: str = liquidator_name
        self.group_name: str = group_name
        self.succeeded: bool = succeeded
        self.signature: str = signature
        self.wallet_address: PublicKey = wallet_address
        self.margin_account_address: PublicKey = margin_account_address
        self.balances_before: typing.List[TokenValue] = balances_before
        self.balances_after: typing.List[TokenValue] = balances_after
        self.changes: typing.List[TokenValue] = TokenValue.changes(balances_before, balances_after)

    def __str__(self) -> str:
        result = "✅" if self.succeeded else "❌"
        changes_text = "\n        ".join([f"{change.value:>15,.8f} {change.token.symbol}" for change in self.changes])
        return f"""« 🥭 Liqudation Event {result} at {self.timestamp}
    💧 Liquidator: {self.liquidator_name}
    🗃️ Group: {self.group_name}
    📇 Signature: {self.signature}
    👛 Wallet: {self.wallet_address}
    💳 Margin Account: {self.margin_account_address}
    💸 Changes:
        {changes_text}
»"""

    def __repr__(self) -> str:
        return f"{self}"


# ✅ Testing

In [ ]:
def _notebook_tests():
    log_level = logging.getLogger().level
    try:
        logging.getLogger().setLevel(logging.CRITICAL)

        list_to_split = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
        split_3 = _split_list_into_chunks(list_to_split, 3)
        assert(len(split_3) == 4)
        assert(split_3[0] == ["a", "b", "c"])
        assert(split_3[1] == ["d", "e", "f"])
        assert(split_3[2] == ["g", "h", "i"])
        assert(split_3[3] == ["j"])
        split_2 = _split_list_into_chunks(list_to_split, 2)
        assert(len(split_2) == 5)
        assert(split_2[0] == ["a", "b"])
        assert(split_2[1] == ["c", "d"])
        assert(split_2[2] == ["e", "f"])
        assert(split_2[3] == ["g", "h"])
        assert(split_2[4] == ["i", "j"])
        split_20 = _split_list_into_chunks(list_to_split, 20)
        assert(len(split_20) == 1)
        assert(split_20[0] == ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"])

        token_lookup = TokenLookup.default_lookups()

        assert(token_lookup.find_by_symbol("ETH").mint == PublicKey("2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk"))
        assert(token_lookup.find_by_mint("AKJHspCwDhABucCxNLXUSfEzb7Ny62RqFtC9uNjJi4fq").symbol == "SRM-SOL")

        market_lookup = SpotMarketLookup.default_lookups()
        assert(market_lookup.find_by_symbol("ETH/USDT").base.symbol == "ETH")
        assert(market_lookup.find_by_symbol("ETH/USDT").quote.symbol == "USDT")
        assert(market_lookup.find_by_symbol("ETH/USDT").address == PublicKey("7dLVkUfBVfCGkFhSXDCq1ukM9usathSgS716t643iFGF"))
        assert(market_lookup.find_by_symbol("BTC/USDC").base.symbol == "BTC")
        assert(market_lookup.find_by_symbol("BTC/USDC").quote.symbol == "USDC")
        assert(market_lookup.find_by_symbol("BTC/USDC").address == PublicKey("A8YFbxQYFVqKZaoYJLLUVcQiWP7G2MeEgW5wsAQgMvFw"))
        assert(market_lookup.find_by_symbol("ETH/BTC") is None)  # No such market
        assert(market_lookup.find_by_address("ByRys5tuUWDgL73G8JBAEfkdFf8JWBzPBDHsBVQ5vbQA").base.symbol == "SRM")
        assert(market_lookup.find_by_address("ByRys5tuUWDgL73G8JBAEfkdFf8JWBzPBDHsBVQ5vbQA").quote.symbol == "USDC")
        assert(market_lookup.find_by_address("ByRys5tuUWDgL73G8JBAEfkdFf8JWBzPBDHsBVQ5vbQA").address == PublicKey("ByRys5tuUWDgL73G8JBAEfkdFf8JWBzPBDHsBVQ5vbQA"))

        balances_before = [
            TokenValue(token_lookup.find_by_symbol("ETH"), Decimal(1)),
            TokenValue(token_lookup.find_by_symbol("BTC"), Decimal("0.1")),
            TokenValue(token_lookup.find_by_symbol("USDT"), Decimal(1000))
        ]
        balances_after = [
            TokenValue(token_lookup.find_by_symbol("ETH"), Decimal(1)),
            TokenValue(token_lookup.find_by_symbol("BTC"), Decimal("0.05")),
            TokenValue(token_lookup.find_by_symbol("USDT"), Decimal(2000))
        ]
        timestamp = datetime.datetime(2021, 5, 17, 12, 20, 56)
        event = LiquidationEvent(timestamp, "Liquidator", "Group", True, "signature",
                                 SYSTEM_PROGRAM_ADDRESS, SYSTEM_PROGRAM_ADDRESS,
                                 balances_before, balances_after)
        assert(str(event) == """« 🥭 Liqudation Event ✅ at 2021-05-17 12:20:56
    💧 Liquidator: Liquidator
    🗃️ Group: Group
    📇 Signature: signature
    👛 Wallet: 11111111111111111111111111111111
    💳 Margin Account: 11111111111111111111111111111111
    💸 Changes:
             0.00000000 ETH
            -0.05000000 BTC
         1,000.00000000 USDT
»""")
    finally:
        logging.getLogger().setLevel(log_level)


_notebook_tests()
del _notebook_tests

# 🏃 Running

In [ ]:
if __name__ == "__main__":
    def _notebook_main():
        log_level = logging.getLogger().level
        try:
            logging.getLogger().setLevel(logging.CRITICAL)

            import base64
            from Context import default_context

            # Data for group BTC_ETH_USDT
            group_3_public_key = PublicKey("7pVYhpKUHw88neQHxgExSH6cerMZ1Axx1ALQP9sxtvQV")
            owner_3_public_key = PublicKey("JD3bq9hGdy38PuWQ4h2YJpELmHVGPPfFSuFkpzAd9zfu")
            encoded_3 = "AwAAAAAAAACCaOmpoURMK6XHelGTaFawcuQ/78/15LAemWI8jrt3SRKLy2R9i60eclDjuDS8+p/ZhvTUd9G7uQVOYCsR6+BhzgEOYK/tsicXvWMZL1QUWj+WWjO7gtLHAp6yzh4ggmR5TYxOe+df4LNiUJGSedvZ1K+r6GIzQEosNxNHhh2V7yAW8uStEyfEUTbEEkKgyDlUOVRWgGFbsiOC/Uzmn5ghfd2vMNvykHBB4JNMAUG0WhTyCizezFE3eOWvscJG7VWUUa5gAAAAAO1Ih8hkuwwAAQAAAAAAAADCuVoJcm8AAAEAAAAAAAAAlFGuYAAAAAAq17B5bBMDAAEAAAAAAAAAmLhxEVn+//8AAAAAAAAAAJRRrmAAAAAA+TEN0IhNFAIBAAAAAAAAAOiJg4cDQlkAAQAAAAAAAACjgEchygfXnwdEo4sw2++jvroovFb2BReD7MwO2ycvyWJ1ExP6tUyAXZHgwFt800+q+x6ZXzsCtRysH3ay+vKU9VYQAHqylxZVK65gEg85g27YuSyvOBZAjJyRmYU9KdCO1D+4ehdPu9dQB1yI1uh75wShdAaFn2o4qrMYwq3SQQIAAAAAAAAAT1DEK0hxpSv5VHH5kTlWeJePlGQpPYZ+uqcUELR4sLKFDy1uAqR6+CTQmradxC1wyyjL+iSft+5XudJWwSdi78PjPEJTOBGbyhEgBwAAAAD1dEYfuWMjjZ/rgK0AAAAAnKsRD43mpS3XCLztBAUAAKp2XItMhJ/N4sYBAAAAAABTEKVFyYfPc+mmjQAAAAAAWCug5qta1bqvFvY5uAAAAACgmZmZmZkZAQAAAAAAAAAAMDMzMzMzMwEAAAAAAAAAS153X9szDlbg9dv9VWFE+e6Hzhj8N5Of9zJLVkbx/U3at9+SEEUZuga7O5tTUrcMDYWDg+LYaAWhSQiN2fYk7UBCDwAAAAAAgI1bAAAAAAAAdDukCwAAAAYGBgICAAAA"
            decoded_3 = base64.b64decode(encoded_3)
            group_3_account_info = AccountInfo(group_3_public_key, False, Decimal(6069120), owner_3_public_key, Decimal(185), decoded_3)
            group_3 = Group.parse(default_context, group_3_account_info)
            print("\n\nThis is hard-coded, not live information!")
            print("3-token group", group_3)

            # Data for group BTC_ETH_SOL_SRM_USDC
            group_5_context = default_context.new_from_group_name("BTC_ETH_SOL_SRM_USDC")
            group_5_public_key = PublicKey("79rqTHePsNnGEX6Re8Xkgf4QEkBfsXncXzyRbyMhHPMU")
            owner_5_public_key = PublicKey("3ZhWpsT19EuBZhG2BvcTVHBtFW4vFacGgLYpXZxotg1P")
            encoded_5 = "AwAAAAAAAAAI9f3BfgBs05rqiAwecBatsPMhUbtwy/lssY5IHfz1as3WedpQtXtJeKZndHMtJOx7NoURsW+VQN5PR7WCMJr+BpuIV/6rgYT7aH9jRhjANdrEOdwa6ztVmKDwAAAAAAF6mkC+0kyNlxFPeKUyBpYL11A4a/pUcOYunlw92EFaYe8xN4hyxY/VqgKDw3gpT/JSVB/N/r9zVgAMlmL27VP2E9HOlNA8BU4JrUJL9JQ30DUtHJTFIXaQN1pfDgz/KVX+5r0fIkHxn8CfTzJ6xolktpxI5iPcYED3iDjXX10LTbrYnfWNWX0g3ojdBQP55TgRaxKmM2rKPCR9ZVj9U6ugOKB1oHM+iXvpy89ra32TRE5LVPCRCxDKQW+HAixGJRL+zsGa6VtOWqJYWqNCV5ef3grP0G7ATNeVtXpsms96wQVUrmAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAFVK5gAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAEAAAAAAAAABVSuYAAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAVUrmAAAAAAPzGbhwwAAAABAAAAAAAAAFtLagYAAAAAAQAAAAAAAAAFVK5gAAAAADi95MmtAAAAAQAAAAAAAADw2EQGAAAAAAEAAAAAAAAAl5zHWv/y5wWxpcFtmFSbBTWMi2Bd8NV+Sr1XVxD132WPgK2ybPoMrgyN4s5T+6J30kS9BrEGnwtaAp4GCD27TVgDGoaPituuWI3sfe9LKjbMNihp3zYxLPyNBlB3cZswVvPrJ9M2SuDyQR+Cyo4YHlucZh37xO6TuuhSDaaRq4hSNqBWdfB5O0wXcsXSAxviIZnpzZb8yVPAJhqxFNViEC+Y1L1+ULx6KHTSRIwn5wXTktPuZEsRtSpUJpqZD9V5y+8i6KVSbQNBUDzf0xeAFH85lOlowgaoBW/MjycjsrXlD0s7jbPTK2SvlK0fgerysq1O8y7bk5yp6001Zr/sLAEAAAAAAAAAAZbGvC2Uwmchkn8Xc2PeT4aBGReM7Kjuqc1DuN5ufbG1vZ7LZ502Gt8wb/WL0dyrOS4eySbPnOiFlVUBjsVtHwAAAAAAAAAA9Mq1OgAAAAAAAAAAAAAAAIDw+gIAAAAAAAAAAAAAAADwf4dkHAAAAAEAAAAAAAAAAMqaOwAAAAAUOu5om0z+/+/n31+gWwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAZwqohV9gfgEAAAAAAAAAAOuuBPiBhIG6j5hOWgMAAAAAoJmZmZmZGQEAAAAAAAAAADAzMzMzMzMBAAAAAAAAAGtqUa92w4LGfIDkG7RkvDMOZX+ssXFAKsKq/DKs+TSjBpU3RtcibIXhdSaNSvboTLAKz91Es3OeaoUjHgSxfAUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABgYJBgkCAgICAAAAAAAAAA=="
            decoded_5 = base64.b64decode(encoded_5)
            group_5_account_info = AccountInfo(group_5_public_key, False, Decimal(9020160), owner_5_public_key, Decimal(135), decoded_5)
            group_5 = Group.parse(group_5_context, group_5_account_info)
            print("\n\nThis is hard-coded, not live information!")
            print("5-token group", group_5)

            token_lookup = TokenLookup.default_lookups()
            print(token_lookup.find_by_symbol("ETH"))
            print(token_lookup.find_by_symbol("BTC"))

            # USDT
            print(token_lookup.find_by_mint(PublicKey("Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB")))

            single_account_info = AccountInfo.load(default_context, default_context.dex_program_id)
            print("DEX account info", single_account_info)

            multiple_account_info = AccountInfo.load_multiple(default_context, [default_context.program_id, default_context.dex_program_id])
            print("Mango program and DEX account info", multiple_account_info)

            balances_before = [
                TokenValue(token_lookup.find_by_symbol("ETH"), Decimal(1)),
                TokenValue(token_lookup.find_by_symbol("BTC"), Decimal("0.1")),
                TokenValue(token_lookup.find_by_symbol("USDT"), Decimal(1000))
            ]
            balances_after = [
                TokenValue(token_lookup.find_by_symbol("ETH"), Decimal(1)),
                TokenValue(token_lookup.find_by_symbol("BTC"), Decimal("0.05")),
                TokenValue(token_lookup.find_by_symbol("USDT"), Decimal(2000))
            ]
            timestamp = datetime.datetime(2021, 5, 17, 12, 20, 56)
            event = LiquidationEvent(timestamp, "Liquidator", "Group", True, "signature",
                                     SYSTEM_PROGRAM_ADDRESS, SYSTEM_PROGRAM_ADDRESS,
                                     balances_before, balances_after)
            print(event)
        finally:
            logging.getLogger().setLevel(log_level)

    _notebook_main()
    del _notebook_main
